In [1]:
with open('input') as f:
    data = [line.strip().split() for line in f]

In [2]:
from collections import Counter
from functools import cached_property, total_ordering


VALUES = "23456789TJQKA"

HAND_RANKS = (
    "High Card",
    "One Pair",
    "Two Pair",
    "Three of a Kind",
    "Full House",
    "Four of a Kind",
    "Five of a Kind",
)


@total_ordering
class CamelCardsHand:
    def __init__(self, cards: str):
        assert len(cards) == 5
        self.cards = cards

    def __repr__(self):
        return f"<CamelCardsHand object ({self})>"
    
    def __str__(self):
        return self.cards

    def __iter__(self):
        return iter(self.cards)

    def __eq__(self, other: "CamelCardsHand"):
        return self.rank_index == other.rank_index

    def __gt__(self, other: "CamelCardsHand"):
        return self.rank_index > other.rank_index

    @cached_property
    def rank(self):
        card_values = Counter([card for card in self])
        
        if len(card_values) == 1:
            return "Five of a Kind"
        if len(card_values) == 2:
            if 4 in card_values.values():
                return "Four of a Kind"
            return "Full House"
        if len(card_values) == 3:
            if 3 in card_values.values():
                return "Three of a Kind"
            return "Two Pair"
        if len(card_values) == 4:
            return "One Pair"
        return "High Card"
    
    @cached_property
    def rank_index(self):
        return HAND_RANKS.index(self.rank)
    
def sort_hand_by_cards(hand):
    return tuple([VALUES.index(card) for card in hand])

In [3]:
all_hands = sorted([
    (CamelCardsHand(hand_str), sort_hand_by_cards(hand_str), int(bid_str))
    for hand_str, bid_str in data
])

In [4]:
print("Part 1:")
print(sum(rank * bid for rank, (hand, srt, bid) in enumerate(sorted(all_hands), start=1)))

Part 1:
253205868


In [5]:
from itertools import combinations_with_replacement

J_REPLACEMENTS = "23456789TQKA"

def get_best_hand_with_joker_replacement(orig_values):
    card_values = Counter(orig_values)
    if "J" in card_values:
        alt_hands = []
        num_jokers = card_values["J"]
        for replacements in combinations_with_replacement(J_REPLACEMENTS, r=num_jokers):
            alt_hand = list(orig_values)
            for replacement in replacements:
                j_pos = alt_hand.index("J")
                alt_hand[j_pos] = replacement
            alt_hand_str = "".join(alt_hand)
            alt_hands.append((CamelCardsHand(alt_hand_str), sort_hand_by_cards(orig_values)))
        
        best = max(alt_hands)
        return best[0]
    return CamelCardsHand(orig_values)

def sort_hand_by_cards_with_joker(hand):
    return tuple([VALUES.index(card) if card != "J" else -1 for card in hand])

In [6]:
all_hands_with_replaced_jokers = sorted([
    (get_best_hand_with_joker_replacement(hand_str), sort_hand_by_cards_with_joker(hand_str), hand_str, int(bid_str))
    for hand_str, bid_str in data
])
print("Part 2:")
print(sum(rank * bid for rank, (hand, srt, hs, bid) in enumerate(all_hands_with_replaced_jokers, start=1)))

Part 2:
253907829
